In [3]:
import re
import csv
with open("Everli_logs/scraper_logs_2025-06-10.csv", "r", encoding="utf-8") as f:
    log_content = f.readlines()
csv_data = [
    ["store_name", "store_id", "num_categories", "num_products", "duration_in_minutes"]
]

store_name = None
store_id = None
num_categories = None
num_products = None
duration_in_minutes = None

for line in log_content:
    if "Start processing Store" in line:
        if store_name is not None:
            csv_data.append([store_name, store_id, num_categories, num_products, duration_in_minutes])
        
        match = re.search(r'Start processing Store (\d+) - (.+)', line)
        if match:
            store_id = match.group(1)
            store_name = match.group(2).strip()
        num_categories = None
        num_products = None
        duration_in_minutes= None
    
    elif "Categories found" in line:
        match = re.search(r'Categories found: (\d+)', line)
        if match:
            num_categories = int(match.group(1))
    
    elif "Appended" in line and "products" in line:
        match = re.search(r'Appended (\d+) products', line)
        if match:
            num_products = int(match.group(1))
    
    elif "duration_in_minutesfor store" in line:
        match = re.search(r'duration_in_minutesfor store \d+: ([\d.]+) min', line)
        if match:
            duration_in_minutes= float(match.group(1))

if store_name is not None:
    csv_data.append([store_name, store_id, num_categories, num_products, duration_in_minutes])

output_csv_path = "everli_store_summary.csv"
with open(output_csv_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

print(f"CSV saved to {output_csv_path}")

CSV saved to everli_store_summary.csv
